In [2]:
from qiskit_gym.envs import LinearFunctionGym, CliffordGym, PermutationGym, LinearFunctionNoisyGym
from qiskit_gym.rl import RLSynthesis, PPOConfig, AlphaZeroConfig, BasicPolicyConfig


from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap
import numpy as np

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,        
    thermal_relaxation_error,)

    #getting the noise model from a fake backe
from qiskit_aer.noise import (
    NoiseModel,
)
from qiskit_ibm_runtime.fake_provider import FakeManhattanV2
backend = FakeManhattanV2()
noise = backend.target['cx']
coupling = backend.coupling_map
model= {}

for pair in coupling:
    model[pair] = (1 - noise[tuple(pair)].error)

print (".")
cmap_6_line = CouplingMap.from_ring(4, bidirectional=True)
env = LinearFunctionNoisyGym.from_coupling_map_noisy(cmap_6_line, basis_gates=["CX", 'SWAP'], noise_model=model, max_depth=128)
print(".")
env2 = LinearFunctionGym.from_coupling_map(cmap_6_line, basis_gates= ['CX', 'SWAP'])

rls = RLSynthesis(env, PPOConfig(), BasicPolicyConfig())
rls.learn(initial_difficulty=3, num_iterations=10, tb_path="runs/lf_4_line_ppo/")  # This will track progress in Tensorboard
print(".")
rls.env.config['gateset']

rls2 = RLSynthesis(env2, PPOConfig(), BasicPolicyConfig())
rls2.learn(num_iterations=10, tb_path="runs/lf_4_line_ppo/")  # This will track progress in Tensorboard


.
.


: 

In [7]:
rls.env.config['gateset']

NameError: name 'rls' is not defined

In [ ]:

from qiskit.circuit.library.generalized_gates import LinearFunction
qc_lf_input = QuantumCircuit(5)

# qc_lf_input.cx(0,3)

# qc_lf_input.cx(0,5)
qc_lf_input.cx(0,2)
qc_lf_input.cx(1,3)
# qc_lf_input.cx(2,4)

# qc_lf_input.cx(6,4)
# qc_lf_input.cx(2,1)
# qc_lf_input.cx(3,1)
# qc_lf_input.cx(0,1)
# qc_lf_input.cx(0,2)
# qc_lf_input.cx(0,3)
# qc_lf_input.cx(0,3)
# qc_lf_input.cx(0,2)
# qc_lf_input.cx(0,3)
qc_lf_input.draw(fold=-1, output="mpl")

In [ ]:
print('Noisy Synthesis')

qc_lf_output = rls.synth(qc_lf_input, num_searches=3000, num_mcts_searches=0, deterministic=False)


# while qc_lf_output is None:
#     qc_lf_output = rls.synth(qc_lf_input, num_searches=1000, num_mcts_searches=0, deterministic=False)

qc_lf_output.draw(fold=-1, output ='mpl',)

In [ ]:
print('not noisy')
qc_lf_output2 = rls2.synth(qc_lf_input, num_searches=2000, num_mcts_searches=0, deterministic=False)
qc_lf_output2.draw(fold=-1, output = 'mpl')

In [ ]:
from qiskit import transpile, generate_preset_pass_manager

from qiskit import QuantumCircuit
# from qiskit_ibm_runtime.fake_provider import FakeAuckland, FakeWashingtonV2
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.transpiler.coupling import CouplingMap
 
# backend = FakeAuckland()
cmap_6_line = CouplingMap.from_ring(5, bidirectional=True)
 

ghz = QuantumCircuit(5)
ghz.cx(0,4)
 
depths = []
pass_manager = generate_preset_pass_manager(
        coupling_map=cmap_6_line,
        optimization_level=1,
        routing_method="sabre",
        layout_method="trivial",  # Fixed layout mapped in circuit order
)

# ghz.draw()
ghz.draw()
pass_manager.run(ghz).draw()


In [ ]:
import random


def generate_random_circ(num_qubits):
    qc = QuantumCircuit(num_qubits)
    for i in range(random.randrange(2, 3)):
        q1 = random.randint(0, num_qubits - 1)
        q2 = q1 + (2 * random.randint(-1, 1)) 
        if q1 == q2 :
            q2 = q1 + 1
        if  q2 >= num_qubits:
            q2 = q1 - 2
        qc.cx(q1,q2)

    return qc
    
    


qc = generate_random_circ(5)
qc.draw(fold=-1, output="mpl")


In [ ]:
# qc_lf_output2.count_ops()
# for val in qc_lf_output2.decompose().data:
#     print(val)

#parrsing data to show 
#make another hash map to represent edges on map
#then add for each op one onto the location in the map
#then multiply by noise factor
#then run this multiple times and find the AVERAGE accross each edge and compare to the noisy version to see if there is a difference in the noise accumulation
map1 = {}
map2 = {}
for _ in range(10):
    # qc_lf_output = rls.synth(qc_lf_input, num_searches=3000, num_mcts_searches=0, deterministic=False)
    qc_input = generate_random_circ(5)
    qc_lf_output = rls.synth(qc_input, num_searches=3000, num_mcts_searches=0, deterministic=False)
    while qc_lf_output is None:
        qc_lf_output = rls.synth(qc_input, num_searches=3000, num_mcts_searches=0, deterministic=False)

    for val in qc_lf_output.decompose().data:
        edge =  sorted([val.qubits[0]._index, val.qubits[1]._index])
        edge = tuple(edge)
        if edge not in map1:
            map1[edge] = 1
        else:
            map1[edge] += 1
    
    qc_lf_output2 = rls2.synth(qc_input, num_searches=1000, num_mcts_searches=0, deterministic=False)
    while qc_lf_output2 is None:
        qc_lf_output2 = rls2.synth(qc_input, num_searches=1000, num_mcts_searches=0, deterministic=False)

    for val in qc_lf_output2.decompose().data:
        edge =  sorted([val.qubits[0]._index, val.qubits[1]._index])
        edge = tuple(edge)
        if edge not in map2:
            map2[edge] = 1
        else:
            map2[edge] += 1

# for val in map2:
#     map2[val] /= 10

print(map1)
print(map2)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sort edges for consistent x-axis
edges = sorted(map1.keys())
x = np.arange(len(edges))

y1 = np.array([map1[e] for e in edges])
y2 = np.array([map2[e] for e in edges])

# Best-fit lines
m1, b1 = np.polyfit(x, y1, 1)
m2, b2 = np.polyfit(x, y2, 1)

plt.figure()

plt.scatter(x, y1)
# plt.plot(x, m1 * x + b1)

plt.scatter(x, y2)
# plt.plot(x, m2 * x + b2)

plt.xlabel("Edge")
plt.ylabel("CNOT Count")
plt.title("CNOT Distribution for (2,3) noisy edge")
plt.xticks(x, [str(e) for e in edges], rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
from qiskit.circuit.library.generalized_gates import LinearFunction
LinearFunction(qc_lf_input) == LinearFunction(qc_lf_output)

In [ ]:
cmap_3_line = CouplingMap.from_line(3, bidirectional=True)
cmap = CouplingMap.from_grid(3,3,bidirectional=True)
env = LinearFunctionNoisyGym.from_coupling_map(cmap)
print(env.config["gateset"])



some_perm = np.random.permutation(9)

rls = RLSynthesis(env, PPOConfig(), BasicPolicyConfig())

rls.env.config["gateset"]

qc_perm = rls.synth(some_perm, num_searches=1000)
qc_perm.draw(fold=-1)


In [ ]:
# You can set a custom state like this (in this case from a circuit)
env.set_state(env.get_state(qc))

env.action_space  # This tells you the number of possible actions (a discrete space of 8 actions)

env.observation_space  # This tells you the type and size of observation space (N by N discrete in this case)

# You can advance the env by providing an action
obs, reward, is_final, _, _ = env.step(2)

# This provides:
# - The observation of the state right after action
# - The reward for that step
# - If we are in a final state

obs, reward, is_final


# One way to do it
env.set_state(env.get_state(qc))
env.render()
for a in [4,2,4]:
    print(f"[{a}] - {env.config['gateset'][a]}")
    obs, reward, is_final, _, _ = env.step(a)
    print(f"[{a}] - Reward: {reward}, Is final: {is_final}")
    env.render()

In [ ]:
cmap_3x3 = CouplingMap.from_grid(3,3, bidirectional=False)
env = PermutationGym.from_coupling_map(cmap_3x3)
rls = RLSynthesis(env, PPOConfig(), BasicPolicyConfig())

rls.env.config["gateset"]

In [ ]:
rls.learn(num_iterations=10, tb_path="runs/perm_square_3x3/")

In [ ]:
#rls.save("models/perm_square_3x3.json", "models/perm_square_3x3.pt")

rls = RLSynthesis.from_config_json("models/perm_square_3x3.json", "models/perm_square_3x3.pt")

In [ ]:
some_perm = np.random.permutation(9)

qc_perm = rls.synth(some_perm, num_searches=1000, num_mcts_searches=0, deterministic=False)
qc_perm.draw(fold=-1)



In [ ]:
qc_perm_input = QuantumCircuit(9)
qc_perm_input.swap(0,8)
qc_perm_input.draw(fold=-1)

In [ ]:
qc_perm_output = rls.synth(qc_perm_input, num_searches=1000, num_mcts_searches=0, deterministic=False)
qc_perm_output.draw(fold=-1)

cmap = CouplingMap.from_heavy_hex(9, bidirectional=True)
env = LinearFunctionNoisyGym.from_coupling_map(cmap)
env.config["gateset"]

In [ ]:
#rls.save("models/lf_5_line.json", "models/lf_5_line.pt")

rls = RLSynthesis.from_config_json("models/lf_5_line.json", "models/lf_5_line.pt")

In [ ]:
qc_lf_input = QuantumCircuit(5)
qc_lf_input.cx(0,4)
qc_lf_input.draw(fold=-1)

In [ ]:
qc_lf_output = rls.synth(qc_lf_input, num_searches=1000, num_mcts_searches=0, deterministic=False)
qc_lf_output.draw(fold=-1)

In [ ]:
from qiskit.circuit.library.generalized_gates import LinearFunction
LinearFunction(qc_lf_input) == LinearFunction(qc_lf_output)

In [ ]:
# Here we want to do Clifford synthesis but we only allow H and S to be placed on qubit 0
env = CliffordGym(
    num_qubits=3, 
    gateset=[
        ("CX", [0,1]),
        ("CX", [1,0]),
        ("CX", [1,2]),
        ("CX", [2,1]),
        ("SWAP", [0,1]),
        ("SWAP", [1,2]),
        ("H", [0]),
        ("S", [0]),
    ]
)
rls = RLSynthesis(env, PPOConfig(), BasicPolicyConfig())
rls.learn(num_iterations=10, tb_path="runs/clifford_3q_custom/")

In [ ]:
#rls.save("models/clifford_3q_custom.json", "models/clifford_3q_custom.pt")

rls = RLSynthesis.from_config_json("models/clifford_3q_custom.json", "models/clifford_3q_custom.pt")

In [ ]:
from qiskit.quantum_info import random_clifford, Clifford

In [ ]:
qc_clifford_in = QuantumCircuit(3)
qc_clifford_in.h(2)

qc_clifford_out = rls.synth(qc_clifford_in, num_searches=1000, num_mcts_searches=0, deterministic=False)
qc_clifford_out.draw(fold=-1)

In [ ]:
some_clifford = random_clifford(3, seed=42)
qc_rand_clifford_out = rls.synth(some_clifford, num_searches=1000, num_mcts_searches=0, deterministic=False)
qc_rand_clifford_out.draw(fold=-1)

In [ ]:
# Equivalent up to phase
np.array_equal(some_clifford.tableau[:,:-1], Clifford(qc_rand_clifford_out).tableau[:,:-1])